In [1]:
import os, sys # add MaaSSim to path (not needed if MaaSSim is already in path)
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import MaaSSim

In [2]:
#!pip install MaaSSim
import MaaSSim.utils
import logging
from dotmap import DotMap
import MaaSSim.simulators
from MaaSSim.data_structures import structures as inData
from MaaSSim.acceptance import f_decline, f_decline_R50
from MaaSSim.acceptance import RA_kpi_veh, RA_kpi_pax

In [3]:
import osmnx as ox
import networkx as nx
import random
import pandas as pd
import matplotlib.pyplot as plt          
import numpy as np
import seaborn as sns

In [4]:
%load_ext autoreload
%autoreload 2

In [34]:
# Do not forget to set the radius for centeral nodes calculation
# The minimum trip distance is considered 1000m and this can be modified through utils.py: generate_demand function. 
# long attribute should be calculated according to the size of city
# Platform fare is 2$ per km

params = MaaSSim.utils.get_config('glance.json')
params.vehicle_fixed_positions = True
params.times.patience = 600 #in second
params.dist_threshold = 10000 #in meter
params.dist_threshold_min = 1000
params.city = "Delft, Netherlands"
params.paths.G = "../../data/graphs/Delft.graphml"
params.paths.skim = "../../data/graphs/Delft.csv"

params.d2d.fuel_cost = 0.25 #euro/km
params.platforms.comm_rate = 0
params.platforms.base_fare = 2
params.platforms.fare = 1.2 
params.platforms.min_fare = 2
params.speeds.ride=10
params.times.decline=20
params.assert_me = False

params.nP = 20
params.nV = 2
params.simTime = 1

In [35]:
sim = MaaSSim.simulators.simulate(params=params, f_driver_decline=f_decline, kpi_veh=RA_kpi_veh, kpi_pax=RA_kpi_pax)

20-05-22 15:12:07-WARNING-Setting up 1h simulation at 2022-05-20 14:44:11 for 2 vehicles and 20 passengers in Delft, Netherlands
20-05-22 15:12:07-INFO--------------------	Starting simulation	-------------------
[0] []
[0] [1]
20-05-22 15:12:07-WARNING-pax    0  got rejected by vehicle 1                2022-05-20 14:44:11
[] [2]
[0] [2]
[] [1]
[1] [1]
[2] []
[2, 3] []
[2, 3, 4] []
[2, 3, 4, 5] []
[2, 3, 4, 5, 6] []
[2, 3, 4, 5, 6] [2]
20-05-22 15:12:07-INFO-pax    0  got to dest at                           2022-05-20 14:57:47
[2, 3, 4, 6, 7] []
20-05-22 15:12:07-INFO-pax    2  has no offers                            2022-05-20 15:00:50
20-05-22 15:12:07-INFO-pax    2  lost his patience and left the system    2022-05-20 15:00:50
[3, 4, 6, 7, 8] []
20-05-22 15:12:07-INFO-pax    3  has no offers                            2022-05-20 15:01:46
20-05-22 15:12:07-INFO-pax    3  lost his patience and left the system    2022-05-20 15:01:46
20-05-22 15:12:07-INFO-pax    4  has no offers       

In [41]:
sim.res[0].pax_exp

,veh_id,WAIT_TIME,nREJECTS,TRAVEL_TIME,LOST_PATIENCE
pax,,,,,
0,14.0,412.0,1.0,314.0,False
1,6.0,589.0,0.0,368.0,False
2,Unfulfilled,615.0,0.0,0.0,True
3,Unfulfilled,615.0,0.0,0.0,True
4,Unfulfilled,615.0,0.0,0.0,True
5,12.0,210.0,0.0,173.0,False
6,6.0,973.0,0.0,164.0,False
7,2.0,585.0,1.0,0.0,True
8,12.0,308.0,0.0,163.0,False


In [40]:
df = pd.DataFrame(sim.pax[2].rides)
df

,pax,pos,t,event,veh_id
0,2,44740049,0,STARTS_DAY,None
1,2,44740049,384,REQUESTS_RIDE,None
2,2,44740049,999,LOSES_PATIENCE,None
3,2,44740049,999,LOSES_PATIENCE,None


In [37]:
df = pd.DataFrame(sim.vehs[2].myrides)
df

,veh,pos,t,event,paxes
0,2,5.052025e+09,0,STARTS_DAY,[]
1,2,5.052025e+09,0,OPENS_APP,[]
2,2,5.052025e+09,30,RECEIVES_REQUEST,[0.0]
3,2,5.052025e+09,30,ACCEPTS_REQUEST,[0.0]
4,2,5.052025e+09,50,IS_ACCEPTED_BY_TRAVELLER,[0.0]
5,2,1.575265e+09,462,ARRIVES_AT_PICKUP,[0.0]
6,2,1.575265e+09,462,MEETS_TRAVELLER_AT_PICKUP,[0.0]
7,2,1.575265e+09,492,DEPARTS_FROM_PICKUP,[0]
8,2,1.413911e+09,806,ARRIVES_AT_DROPOFF,[0]
9,2,1.413911e+09,816,RECEIVES_REQUEST,[5.0]
